In [ ]:
images_path = 'Dataset/train_images'

In [ ]:
import os

In [ ]:
import pandas as pd 

In [ ]:
import numpy as np 

In [ ]:
import plotly.express as px

In [ ]:
import torch

In [ ]:
import torch.nn as nn

In [ ]:
import torch.optim as optim

In [ ]:
import torch.nn.functional as F

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torchvision import transforms, models

In [ ]:
from IPython.display import display

In [ ]:
import albumentations as albu

In [ ]:
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from collections import Counter

In [ ]:
import pdb

In [ ]:
import seaborn as sns

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [ ]:
train_df = pd.read_csv('Dataset/train.csv', nrows=1000)

In [ ]:
display(train_df.head())

In [ ]:
display(train_df.sample(10))

In [ ]:
def split_image_label(x):
    res = x.split('_')
    return (x, res[0], res[1])

In [ ]:
image_label_pairs = []
for row in train_df.iterrows():
    image_label_pairs.append(split_image_label(row[1].Image_Label))

In [ ]:
train_df = pd.merge(train_df, 
                    pd.DataFrame(image_label_pairs, columns=['Image_Label', 'Image', 'Label']), 
                    on=['Image_Label'])

train_df.head()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df['Label'].value_counts()

In [ ]:
px.pie(train_df, values=train_df['Label'].value_counts().values, names=train_df['Label'].value_counts().index)

In [ ]:
px.bar(train_df, y=train_df['Label'].value_counts().values, x=train_df['Label'].value_counts().index)

In [ ]:
train_df['Label'] = train_df['Label'].astype('category')

In [ ]:
train_df['Label_Encoded'] = train_df['Label'].cat.codes

display(train_df.head())

train_df.to_csv('train_encoded.csv', index=False)

train_image_ids = train_df['Image'].unique()

In [ ]:
train_mask_df = train_df.pivot(index='Image', columns='Label', values='EncodedPixels')

In [ ]:
train_mask_df = pd.DataFrame(train_mask_df.to_records()).set_index('Image')

In [ ]:
display(train_mask_df.head(2))

In [ ]:
train_mask_df.to_csv('train_mask.csv', index=True)

In [ ]:
train_label_df = train_mask_df.notnull().reset_index(drop=True).values

In [ ]:
print(train_label_df[:5])

In [ ]:
print(train_mask_df.shape)
print(train_label_df.shape)

In [ ]:
X_train, y_train, X_valid, y_valid = iterative_train_test_split(train_mask_df.reset_index().values, train_label_df, test_size = 0.2)

In [ ]:
count_org = Counter(combination for row in get_combination_wise_output_matrix(train_label_df, order=2) for combination in row)

In [ ]:
count_train = Counter(combination for row in get_combination_wise_output_matrix(y_train, order=2) for combination in row)

In [ ]:
count_valid = Counter(combination for row in get_combination_wise_output_matrix(y_valid, order=2) for combination in row)

In [ ]:
columns = train_mask_df.reset_index().columns

In [ ]:
X_train = pd.DataFrame(X_train, columns=columns)

In [ ]:
X_train = X_train.set_index('Image')

In [ ]:
X_valid = pd.DataFrame(X_valid, columns=columns).set_index('Image')

In [ ]:
display(X_train.head(2))

In [ ]:
display(X_valid.head(2))

In [ ]:
pd.DataFrame({'org': Counter(str(combination) for row in get_combination_wise_output_matrix(train_label_df, order=2) for combination in row),
              'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_train, order=2) for combination in row),
             'valid': Counter(str(combination) for row in get_combination_wise_output_matrix(y_valid, order=2) for combination in row)
                }).plot.bar()

In [ ]:
for col in X_train.columns:
    print('{:8s}: X_train = {:,}, X_valid = {:,}'.format(col,
                                             X_train[col].notnull().sum(),
                                             X_valid[col].notnull().sum()))

In [ ]:
class AutoEncoderNet(nn.Module):
    def __init__(self, in_channels, down_layer_channels, bottom_channels, up_layer_channels, out_channels=1):
        super().__init__()
        
        self.down_layers = self.init_down_layers(in_channels,
                                                down_layer_channels)
        
        self.bottom_layer = self.init_bottom_layer(in_channels=down_layer_channels[-1],
                                                  out_channels=bottom_channels)
        
        self.up_layers = self.init_up_layers(in_channels=bottom_channels,
                                            up_layer_channels=up_layer_channels,
                                            down_layer_channels=down_layer_channels)
        
        self.output_layer = self.init_output_layer(in_channels=up_layer_channels[0],
                                                  out_channels=out_channels)
        
    def init_down_layers(self, in_channels, down_layer_channels):
        layers = nn.ModuleList()
        
        ch_in = in_channels
        for ch_out in down_layer_channels:
            layers.append(AutoFall(in_channels=ch_in,
                                     out_channels=ch_out))
            
            ch_in = ch_out
        
        return layers
    def init_bottom_layer(self, in_channels, out_channels):
        return ResnetBlock(in_channels, out_channels)
    
    def init_up_layers(self, in_channels, up_layer_channels, down_layer_channels):
        layers = nn.ModuleList()
        
        ch_in = in_channels
        for ch_out, ch_middle in zip(reversed(up_layer_channels), reversed(down_layer_channels)):
            layers.append(AutoRise(ch_in, ch_middle, out_channels=ch_out))
            ch_in = ch_out
        
        return layers
    def init_output_layer(self, in_channels, out_channels, kernel_size=3, padding=1):
        
        return nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, padding=padding),
                             nn.ReLU(),
                             nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding),
                            )
    def forward(self, X):
        y = X
        
        y_middles = []
        for layer in self.down_layers:
            y, y_m = layer(y)
            y_middles.append(y_m)
            
        y = self.bottom_layer(y)
        
        for index, (layer, y_m) in enumerate(zip(self.up_layers, reversed(y_middles))):
            y = layer(y, X_middle=y_m)
        
        y = self.output_layer(y)
        
        return y

In [ ]:
class AutoRise(nn.Module):
    def __init__(self, in_up_features, in_middle_features, out_channels):
        super().__init__()
        
        self.conv = ResnetBlock(in_channels=(in_up_features+in_middle_features), 
                                out_channels=out_channels)
        
    def forward(self, X, X_middle):
#         print('X:{}, X_middle:{}'.format(X.shape,
#                                         X_middle.shape))
              
        y = F.interpolate(X, scale_factor=2, mode='bilinear', align_corners=True)
        y = torch.cat([y, X_middle], dim=1)
        
        y = self.conv(y)
        
        return y

In [ ]:
class AutoFall(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = ResnetBlock(in_channels, out_channels)
        
    def forward(self, X):
        y = self.conv(X)
        y_middle = y
        
        y = F.max_pool2d(y, kernel_size=2)
        
        return y, y_middle

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding)
        self.conv3 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding)
        
    def forward(self, X):
        
        y = F.relu(self.conv1(X))
        y_direct = y
        
        y = F.relu(self.conv2(y))
        y = y_direct + F.relu(self.conv3(y))
        return y

In [ ]:
def rle_to_mask(rle_string, width, height):   
    rows, cols = height, width
    
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
        rle_pairs = np.array(rle_numbers).reshape(-1,2)
        img = np.zeros(rows*cols, dtype=np.uint8)
        for index, length in rle_pairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, filenames, root, transform=None, to_tensor=False):
        super().__init__()
        self.filenames = filenames
        self.root = root
        self.transform = transform
        self.to_tensor = to_tensor
        
        self.tensor = transforms.ToTensor()
        
    
    def __len__(self):
        return len(self.filenames)
    
    
    def __getitem__(self, index):
        filename = self.filenames[index]
        filename = os.path.join(self.root, filename)
        image = np.array(Image.open(filename))
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
            
        if self.to_tensor is True:
            image = self.tensor(image)
            
        return image

In [ ]:
class MaskDataset(Dataset):
    def __init__(self, shape_type, image_ids, mask_df, width, height):
        super().__init__()
        self.shape_type = shape_type
        self.image_ids = image_ids
        self.mask_df = mask_df
        self.width = width
        self.height = height
        
    def __len__(self):
        return len(self.image_ids)
    
    
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        
        rle = self.mask_df.loc[image_id, self.shape_type]
        
        
        if isinstance(rle, str):
            mask_image = rle_to_mask(rle, width=self.width, height=self.height)
            mask_image = np.expand_dims(mask_image, axis=-1)
        else:
            mask_image = np.zeros((self.height, self.width, 1))
            
        return mask_image
    
    @property
    def label(self):
        return self.shape_type
    
    @property
    def labels(self):
        return self.mask_df.columns.to_list()

In [ ]:
class MultiMaskDataset(Dataset):
    def __init__(self, image_ids, mask_df, width, height):
        super().__init__()
        self.image_ids = image_ids
        self.mask_df = mask_df
        self.width = width
        self.height = height
        
        
    def __len__(self):
        return len(self.image_ids)
    
    
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        
        rle_masks = self.mask_df.loc[image_id]
        
        masks = None
        for shape_type, rle in rle_masks.items():
#             print(type(rle))
            if isinstance(rle, str):
                mask_image = rle_to_mask(rle, width=self.width, height=self.height)
                mask_image = np.expand_dims(mask_image, axis=-1)
            else:
                mask_image = np.zeros((self.height, self.width, 1))
            
            if masks is None:
                masks = mask_image
            else:
                masks = np.concatenate((masks, mask_image), axis=-1)
            
        return masks
    
    @property
    def labels(self):
        return self.mask_df.columns.to_list()
    
    


In [ ]:
class MaskLabelDataset(Dataset):
    def __init__(self, image_ids, mask_df, to_tensor=False):
        self.image_ids = image_ids
        self.mask_df = mask_df
        self.to_tensor = to_tensor
        
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        rle_masks = self.mask_df.loc[image_id]
        
        labels = []
        for shape_type, rle in rle_masks.items():
            if isinstance(rle, str):
                labels.append(1)
            else:
                labels.append(0)
        
        if self.to_tensor is True:
            labels = torch.tensor(labels)
            
            
        return labels
    
    @property
    def labels(self):
        return self.mask_df.columns.to_list()

In [ ]:
class ImageMaskDataset(Dataset):
    def __init__(self, image_dataset: ImageDataset, 
                     mask_dataset,
                    transform=None,
                    to_tensor=True):
        
        super().__init__()
        self.image_dataset = image_dataset
        self.mask_dataset = mask_dataset
        self.transform = transform
        self.to_tensor = to_tensor
        self.tensor = transforms.ToTensor()
        
        
    def __len__(self):
        return len(self.image_dataset)
        
    def __getitem__(self, index):
        image = self.image_dataset[index]
        masks = self.mask_dataset[index]
        
        if self.transform is not None:
            data = {'image': image, 'mask': masks}
            
            res = self.transform(**data)
            
            image = res['image']
            masks = res['mask']
            
        if self.to_tensor is True:
            image, masks = self.tensor(image), self.tensor(masks)
        
        return image, masks
    
    @property
    def labels(self):
        return self.mask_dataset.labels

In [ ]:
image_dataset = ImageDataset(X_train.index, root=images_path)

In [ ]:
mask_dataset = MultiMaskDataset(X_train.index,
                           train_mask_df,
                           width=2100,
                           height=1400)

In [ ]:
masks = mask_dataset[0]

In [ ]:
transform = albu.Compose([albu.Resize(224, 224, always_apply=True),
                         ])


In [ ]:
cloud_dataset = ImageMaskDataset(image_dataset, mask_dataset, transform=transform, to_tensor=False)

In [ ]:
image, masks = cloud_dataset[0]

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
axes[0].imshow(image)

for index, label in enumerate(cloud_dataset.labels):
    mask = masks[:, :, index]
    axes[index+1].imshow(mask)
    axes[index+1].set_title(label)

In [ ]:
class ImageLabelDataset(Dataset):
    def __init__(self, image_dataset: ImageDataset,
                        mask_label_dataset: MaskLabelDataset):
        super().__init__()
        self.image_dataset = image_dataset
        self.mask_label_dataset = mask_label_dataset
        
        
    def __len__(self):
        return len(self.image_dataset)
    
    def __getitem__(self, index):
        image, label = self.image_dataset[index], self.mask_label_dataset[index]
        
        return image, label
        
    

In [ ]:
batch_size = 8

device = 'cpu'

In [ ]:
def get_train_dataloader(shape_type, image_ids, train_mask_df, path_root, batch_size, to_tensor=True):
    transform = albu.Compose([albu.HorizontalFlip(),
                                    albu.OneOf([
                                        albu.RandomContrast(),
                                        albu.RandomGamma(),
                                        albu.RandomBrightness(),
                                        ], p=0.3),
                                    albu.OneOf([
                                        albu.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                                        albu.GridDistortion(),
                                        albu.OpticalDistortion(distort_limit=2, shift_limit=0.5),
                                        ], p=0.3),
                                    albu.Resize(512, 512, always_apply=True)
                                    ])

    image_dataset = ImageDataset(image_ids,
                                    root=path_root,
                                    to_tensor=False)

    mask_dataset = MaskDataset(shape_type,
                                image_ids,
                               train_mask_df,
                               width=2100,
                               height=1400)
    
    image_mask_dataset = ImageMaskDataset(image_dataset,
                                            mask_dataset,
                                            transform=transform,
                                            to_tensor=to_tensor)
    
    loader = DataLoader(dataset=image_mask_dataset,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=0,
                             pin_memory=True,
                             drop_last=True)
    
    
    return loader

In [ ]:
def get_valid_dataloader(shape_type, image_ids, train_mask_df, path_root, batch_size, to_tensor=True):
    
    transform = albu.Compose([albu.Resize(512, 512, always_apply=True)
                                    ])

    image_dataset = ImageDataset(image_ids,
                                root=path_root,
                                to_tensor=False)

    mask_dataset = MaskDataset(shape_type,
                               image_ids,
                               train_mask_df,
                               width=2100,
                               height=1400)
    
    image_mask_dataset = ImageMaskDataset(image_dataset,
                                        mask_dataset,
                                        transform=transform,
                                         to_tensor=to_tensor)
    
    loader = DataLoader(dataset=image_mask_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=0,
                             pin_memory=True,
                             drop_last=True)
    
    return loader

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    
    running_loss = torch.tensor(0.0).to(device)
    batch_num = 0
    print(dataloader)
    for images, masks in dataloader:
        
        images, masks = images.to(device), masks.to(device, dtype=torch.float)
        
        y_preds = model(images)
        
        loss = criterion(y_preds, masks)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.detach()
        batch_num += 1
    return running_loss.to('cpu').item() / batch_num

In [ ]:
@torch.no_grad()
def evaluate(model, dataloader, criterion, device):
    model.eval()
    
    running_loss = torch.tensor(0.0).to(device)
    batch_num = 0
    
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        y_preds = model(images)
        
        loss = criterion(y_preds, masks)
        
        running_loss += loss.detach()
        batch_num += 1
    return running_loss.to('cpu').item() / batch_num

In [ ]:
model = AutoEncoderNet(in_channels=3,
             out_channels=1,
             down_layer_channels=[32, 64, 128, 256],
             bottom_channels=512,
             up_layer_channels=[32, 64, 128, 256])

In [ ]:
epochs = 10

In [ ]:
drop_dict = {}

In [ ]:
for shape_type in train_mask_df.columns:
    filename_model = 'model_classifier_{}.pth'.format(shape_type)

    model = AutoEncoderNet(in_channels=3,
             out_channels=1,
             down_layer_channels=[32, 64, 128, 256],
             bottom_channels=512,
             up_layer_channels=[32, 64, 128, 256])
    

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.BCEWithLogitsLoss() # multi-label classification
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       patience=3,
                                                       mode='min',
                                                       verbose=True)

    dropping = []
    min_loss = 99999.0
    model = model.to(device)

    image_ids = X_train[X_train[shape_type].notnull()].index
    train_loader = get_train_dataloader(shape_type,
                                        image_ids, 
                                        train_mask_df, 
                                        images_path,
                                       batch_size,
                                       to_tensor=True)

    image_ids = X_valid[X_valid[shape_type].notnull()].index
    valid_loader = get_valid_dataloader(shape_type,
                                        image_ids, 
                                        train_mask_df, 
                                        images_path,
                                       batch_size,
                                       to_tensor=True)


    for i_epoch in tqdm_notebook(range(epochs)):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        lr_scheduler.step(train_loss)

        valid_loss = evaluate(model, valid_loader, criterion, device)

        dropping.append((train_loss, valid_loss))

        print('Epoch {}: train loss={}, valid loss={}'.format(i_epoch,
                                                             train_loss,
                                                             valid_loss))


        if train_loss < min_loss:
            min_loss = train_loss

            print('Save model')
            torch.save(model.state_dict(), filename_model)
        


    drop_dict[shape_type] = dropping
    
    pd.DataFrame(dropping, columns=['train', 'valid']).plot.line()


In [ ]:
dfx = pd.DataFrame(dropping, columns=['train', 'valid'])

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for index, (shape_type, dropping) in enumerate(drop_dict.items()):
    ax = axes[index]
    ax = dfx.plot.line(ax=ax)
    ax.set_title(shape_type)